For each GPS location seen in the dataset, a set of tags can be generated from Open Street Maps that will give us more context about where the person may have been going. 


We can collect these tags, and classify the GPS location as one of (Commerical, Industrial, Residential, School, Transportation etc.) so that the location category can make trip features more insightful and easier to categorise as commute, leisure, chores, or some other type of trip.

One method is to analyse the start and end points of a given trip. If these locations can be categorised as - home, work, commercial unit (shops, cafes, restaurants, retail parks), school, tourist location and all others in one category; it gives a good starting point to start clubbing the trips. This can be done by taking the GPS location of the start/end or any other significant halts during the trip and mapping it to the nearest landmark.  

In [44]:
import pandas as pd
import re
import numpy as np

df = pd.read_csv('data/processed/derived_route_features_v2.csv').iloc[:, 1:]  # Removing index column
df 




,trip_id,distance,duration,avg_speed,start_time,end_time,start_location,end_location,halt_count,avg_halt_duration,max_halt_duration,max_halt_location
0,C001_06-Veloci2_1.0,5080.199201,19.950000,4.244110,2024-01-31 17:49:44,2024-01-31 18:09:41,"(52.66710166666667, -8.576183333333333)","(52.67005833333333, -8.633385)",6,33.791667,96.0,"(52.66534, -8.607245)"
1,C001_06-Veloci2_110.0,6298.928335,19.500000,5.383699,2024-03-04 08:42:54,2024-03-04 09:02:24,"(52.67011, -8.633483333333333)","(52.667611666666666, -8.574975)",3,7.636364,20.0,"(52.66794166666666, -8.576695)"
2,C001_06-Veloci2_111.0,1765.016288,9.516667,3.091097,2024-03-04 10:18:56,2024-03-04 10:28:27,"(52.666605, -8.57684)","(52.672138333333336, -8.556015)",3,86.214286,309.0,"(52.672138333333336, -8.556015)"
3,C001_06-Veloci2_112.0,3399.361455,7.916667,7.156550,2024-03-04 12:21:25,2024-03-04 12:29:20,"(52.67209666666667, -8.555928333333334)","(52.671821666666666, -8.55602)",3,188.461538,451.0,"(52.671821666666666, -8.55602)"
4,C001_06-Veloci2_113.0,8880.359707,33.950000,4.359529,2024-03-04 18:16:24,2024-03-04 18:50:21,"(52.671821666666666, -8.55602)","(52.66999833333333, -8.63372)",8,41.325000,223.0,"(52.670008333333335, -8.633391666666666)"
...,...,...,...,...,...,...,...,...,...,...,...,...
4653,P666_ZZ-JG_86.0,4132.682274,21.433333,3.213594,2023-09-20 15:43:41,2023-09-20 16:05:07,"(52.67801833333333, -8.569161666666666)","(52.666845, -8.543201666666667)",4,160.655172,316.0,"(52.666845, -8.543201666666667)"
4654,P666_ZZ-JG_87.0,3149.144739,21.216667,2.473798,2023-09-20 16:22:05,2023-09-20 16:43:18,"(52.666205, -8.54393)","(52.66678666666667, -8.543111666666666)",5,119.760000,281.0,"(52.65831333333333, -8.54979)"
4655,P666_ZZ-JG_88.0,3692.867836,24.033333,2.560935,2023-09-20 17:28:30,2023-09-20 17:52:32,"(52.66665833333333, -8.543538333333334)","(52.66679666666667, -8.543175)",5,211.750000,581.0,"(52.66080333333333, -8.54976)"
4656,P666_ZZ-JG_89.0,3149.814093,16.933333,3.100211,2023-09-20 18:53:16,2023-09-20 19:10:12,"(52.66588166666666, -8.545276666666666)","(52.666855, -8.543191666666667)",6,72.350000,365.0,"(52.65833666666666, -8.549711666666667)"


In [45]:
df

,trip_id,distance,duration,avg_speed,start_time,end_time,start_location,end_location,halt_count,avg_halt_duration,max_halt_duration,max_halt_location
0,C001_06-Veloci2_1.0,5080.199201,19.950000,4.244110,2024-01-31 17:49:44,2024-01-31 18:09:41,"(52.66710166666667, -8.576183333333333)","(52.67005833333333, -8.633385)",6,33.791667,96.0,"(52.66534, -8.607245)"
1,C001_06-Veloci2_110.0,6298.928335,19.500000,5.383699,2024-03-04 08:42:54,2024-03-04 09:02:24,"(52.67011, -8.633483333333333)","(52.667611666666666, -8.574975)",3,7.636364,20.0,"(52.66794166666666, -8.576695)"
2,C001_06-Veloci2_111.0,1765.016288,9.516667,3.091097,2024-03-04 10:18:56,2024-03-04 10:28:27,"(52.666605, -8.57684)","(52.672138333333336, -8.556015)",3,86.214286,309.0,"(52.672138333333336, -8.556015)"
3,C001_06-Veloci2_112.0,3399.361455,7.916667,7.156550,2024-03-04 12:21:25,2024-03-04 12:29:20,"(52.67209666666667, -8.555928333333334)","(52.671821666666666, -8.55602)",3,188.461538,451.0,"(52.671821666666666, -8.55602)"
4,C001_06-Veloci2_113.0,8880.359707,33.950000,4.359529,2024-03-04 18:16:24,2024-03-04 18:50:21,"(52.671821666666666, -8.55602)","(52.66999833333333, -8.63372)",8,41.325000,223.0,"(52.670008333333335, -8.633391666666666)"
...,...,...,...,...,...,...,...,...,...,...,...,...
4653,P666_ZZ-JG_86.0,4132.682274,21.433333,3.213594,2023-09-20 15:43:41,2023-09-20 16:05:07,"(52.67801833333333, -8.569161666666666)","(52.666845, -8.543201666666667)",4,160.655172,316.0,"(52.666845, -8.543201666666667)"
4654,P666_ZZ-JG_87.0,3149.144739,21.216667,2.473798,2023-09-20 16:22:05,2023-09-20 16:43:18,"(52.666205, -8.54393)","(52.66678666666667, -8.543111666666666)",5,119.760000,281.0,"(52.65831333333333, -8.54979)"
4655,P666_ZZ-JG_88.0,3692.867836,24.033333,2.560935,2023-09-20 17:28:30,2023-09-20 17:52:32,"(52.66665833333333, -8.543538333333334)","(52.66679666666667, -8.543175)",5,211.750000,581.0,"(52.66080333333333, -8.54976)"
4656,P666_ZZ-JG_89.0,3149.814093,16.933333,3.100211,2023-09-20 18:53:16,2023-09-20 19:10:12,"(52.66588166666666, -8.545276666666666)","(52.666855, -8.543191666666667)",6,72.350000,365.0,"(52.65833666666666, -8.549711666666667)"


In [46]:
import pandas as pd
from collections import Counter
# from tqdm import tqdm
import osmnx as ox


In [ ]:

def osm_info_from_point(loc, radius=20):
    lat, lon = loc
    tags = {
        'landuse': True, 'building': True, 'amenity': True, 'highway': True,
        'tourism': True, 'shop': True, 'natural': True, 'leisure': True,
        'man_made': True, 'office': True, 'place': True, 'public_transport': True
    }
    try:
        gdf = ox.features_from_point((lat, lon), tags=tags, dist=radius)
        if gdf.empty:
            label = 'Unknown'
        else:
            label = gdf

        return {'latitude': lat, 'longitude': lon, 'label': label}

    except Exception as e:
        return {'latitude': lat, 'longitude': lon, 'label': f'Error: {e}'}



In [8]:

def get_unique_location_labels(features_df, radius=20, cache_file="osm_cache.csv"):
    """
    Efficiently fetch OSM labels only for unique coordinates.
    """
    # Extract all locations
    all_points = (
        list(features_df['start_location']) +
        list(features_df['end_location']) +
        list(features_df['max_halt_location'])
    )
    
    unique_points = list(set(all_points))
    print(f"Found {len(unique_points)} unique coordinates.")

    # If cache exists, load it
    if os.path.exists(cache_file):
        cache_df = pd.read_csv(cache_file)
        done_points = set(zip(cache_df.latitude, cache_df.longitude))
        print(f"Loaded cache with {len(done_points)} existing points.")
    else:
        cache_df = pd.DataFrame(columns=['latitude', 'longitude', 'label'])
        done_points = set()

    # Find remaining points
    to_do = [pt for pt in unique_points if pt not in done_points]
    print(f"Need to process {len(to_do)} new points.")

    results = []
    for loc in tqdm(to_do, desc="Fetching OSM labels"):
        res = osm_info_from_point(loc, radius)
        results.append(res)
        # Save progress incrementally
        if len(results) % 50 == 0:
            temp = pd.DataFrame(results)
            cache_df = pd.concat([cache_df, temp], ignore_index=True)
            cache_df.to_csv(cache_file, index=False)
            results = []

    # Save remaining
    if results:
        temp = pd.DataFrame(results)
        cache_df = pd.concat([cache_df, temp], ignore_index=True)
        cache_df.to_csv(cache_file, index=False)

    return cache_df




In [9]:

def assign_labels_with_cache(features_df, cache_df):
    lookup = {
        (row.latitude, row.longitude): row.label
        for _, row in cache_df.iterrows()
    }

    def get_label(loc):
        return lookup.get(loc, 'Not found')

    features_df = features_df.copy()
    features_df['start_label'] = features_df['start_location'].apply(get_label)
    features_df['end_label'] = features_df['end_location'].apply(get_label)
    features_df['max_halt_label'] = features_df['max_halt_location'].apply(get_label)

    return features_df



In [10]:

import math
import os

def process_in_chunks(features_df, chunk_size=1000, cache_file='osm_cache.csv', out_dir='results'):
    os.makedirs(out_dir, exist_ok=True)
    n_chunks = math.ceil(len(features_df) / chunk_size)

    for i in range(n_chunks):
        out_path = f"{out_dir}/raw_tags_chunk{i}.csv"
        if os.path.exists(out_path):
            print(f"Skipping chunk {i} (already done).")
            continue

        chunk = features_df.iloc[i*chunk_size:(i+1)*chunk_size]
        cache_df = get_unique_location_labels(chunk, cache_file=cache_file)
        labelled_chunk = assign_labels_with_cache(chunk, cache_df)
        labelled_chunk.to_csv(out_path, index=False)
        print(f"Saved chunk {i} → {out_path}")





In [47]:
features_df = df

In [48]:

features_df['start_location'] = features_df['start_location'].apply(tuple)
features_df['end_location'] = features_df['end_location'].apply(tuple)
features_df['max_halt_location'] = features_df['max_halt_location'].apply(tuple)

process_in_chunks(features_df, chunk_size=100)


Skipping chunk 0 (already done).
Skipping chunk 1 (already done).
Skipping chunk 2 (already done).
Skipping chunk 3 (already done).
Skipping chunk 4 (already done).
Skipping chunk 5 (already done).
Skipping chunk 6 (already done).
Skipping chunk 7 (already done).
Skipping chunk 8 (already done).
Skipping chunk 9 (already done).
Skipping chunk 10 (already done).
Skipping chunk 11 (already done).
Skipping chunk 12 (already done).
Skipping chunk 13 (already done).
Skipping chunk 14 (already done).
Skipping chunk 15 (already done).
Skipping chunk 16 (already done).
Skipping chunk 17 (already done).
Skipping chunk 18 (already done).
Skipping chunk 19 (already done).
Skipping chunk 20 (already done).
Skipping chunk 21 (already done).
Skipping chunk 22 (already done).
Skipping chunk 23 (already done).
Skipping chunk 24 (already done).
Skipping chunk 25 (already done).
Skipping chunk 26 (already done).
Skipping chunk 27 (already done).
Skipping chunk 28 (already done).
Skipping chunk 29 (alrea

In [49]:
osm_data = pd.read_csv('results/merged_trip_data.csv')

In [50]:
import re
from collections import Counter

def classify_by_frequency(df, label_name):
    
    text = df[label_name]
    words = re.findall(r"\b(?=\w*[A-Za-z])[\w:]+\b", text)

    exclude = {"POLYGON", "LINESTRING", "NaN", "element", 'way', 'geometry', 'id', 'Limerick'}
    words = [w for w in words if w not in exclude]
    
    # count frequencies
    tag_counts = Counter(words)
    # print(tag_counts)
    # Now define category keywords
    category_keywords = {
        'Commercial': ['service', 'restaurant', 'cafe', 'fast_food', 'bar', 'pub', 'shop', 'supermarket', 'retail', 'Park',
                        'commercial', 'office', 'parking', 'shop', 'market', 'mall'],
        'Residential': ['service', 'residential', 'apartments', 'house', 'terrace', 'bungalow', 'residence', 'flat', 'mansion', 'condo'],
        'Industrial': ['service', 'industrial', 'factory', 'works', 'offices', 'workspace', 'parking'],
        'School': ['service','school', 'library', 'college', 'university', 'montessori', 'parking', 'academy', 'educational', 'education', 'academic', 'research'],
        'Public Institution': ['service','hospital', 'clinic', 'parking', 'townhall', 'government', 'police', 'fire_station'],
        'Recreational': ['service', 'garden', 'playground', 'pitch', 'leisure', 'forest', 'tourism', 'park', 'hotel'],
        'Religious': ['service', 'church', 'mosque', 'temple', 'place_of_worship'],
        'Transportation': ['service', 'road', 'railway', 'aeroway', 'bus_stop', 'traffic', 'toll', 'junction', 'lane', 'parking_space', 'parking_aisle']
    }
    
    # Score each category by summing occurrences of matching keywords
    scores = {cat: 0 for cat in category_keywords}
    for cat, keywords in category_keywords.items():
        for word, count in tag_counts.items():
            if any(kw in word for kw in keywords):
                scores[cat] += count
    
    # print(scores)
    # Pick the category with the highest score
    max_score = max(scores.values())
    top_cats = [cat for cat, score in scores.items() if score == max_score]
    
    # Priority for tie-breaker
    priority = ['School', 'Public Institution', 'Residential','Industrial', 'Commercial', 'Recreational', 'Transportation', 'Religious']
    for cat in priority:
        if cat in top_cats:
            return cat
    
    return 'Unknown'


In [51]:
labeled_categories = []

for i in range(0, len(osm_data)):
    category = classify_by_frequency(osm_data.iloc[i], 'start_label')
    labeled_categories.append(category)
osm_data['start_category'] = labeled_categories
labeled_categories = []
for i in range(0, len(osm_data)):
    category = classify_by_frequency(osm_data.iloc[i], 'end_label')
    labeled_categories.append(category)
osm_data['end_category'] = labeled_categories

labeled_categories = []
for i in range(0, len(osm_data)):
    category = classify_by_frequency(osm_data.iloc[i], 'max_halt_label')
    labeled_categories.append(category)
osm_data['max_halt_category'] = labeled_categories
osm_data


,trip_id,distance,duration,avg_speed,start_time,end_time,start_location,end_location,halt_count,avg_halt_duration,max_halt_duration,max_halt_location,start_label,end_label,max_halt_label,start_label_manual,start_category,end_category,max_halt_category
0,C001_06-Veloci2_1.0,5080.19,19.950000,4.244102,31/01/2024 17:49,31/01/2024 18:09,"(52.66710166666667, -8.576183333333333)","(52.67005833333333, -8.633385)",6,33.791667,96.0,"(52.66534, -8.607245)",...,...,...,Commercial,Commercial,Residential,Commercial
1,C001_06-Veloci2_110.0,6298.96,19.500000,5.383726,04/03/2024 08:42,04/03/2024 09:02,"(52.67011, -8.633483333333333)","(52.667611666666666, -8.574975)",3,7.636364,20.0,"(52.66794166666666, -8.576695)",...,...,...,Residential,Residential,Commercial,Residential
2,C001_06-Veloci2_111.0,1765.01,9.516667,3.091086,04/03/2024 10:18,04/03/2024 10:28,"(52.666605, -8.57684)","(52.672138333333336, -8.556015)",3,86.214286,309.0,"(52.672138333333336, -8.556015)",...,...,...,Recreational,Recreational,Commercial,Commercial
3,C001_06-Veloci2_112.0,3391.89,7.916667,7.140821,04/03/2024 12:21,04/03/2024 12:29,"(52.67209666666667, -8.555928333333334)","(52.671821666666666, -8.55602)",3,188.461538,451.0,"(52.671821666666666, -8.55602)",...,...,...,Residential,Commercial,Commercial,Commercial
4,C001_06-Veloci2_113.0,8880.32,33.950000,4.359509,04/03/2024 18:16,04/03/2024 18:50,"(52.671821666666666, -8.55602)","(52.66999833333333, -8.63372)",8,41.325000,223.0,"(52.670008333333335, -8.633391666666666)",...,...,...,Residential,Commercial,Residential,Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4653,P666_ZZ-JG_86.0,4132.69,21.433333,3.213600,2023-09-20 15:43:41,2023-09-20 16:05:07,"(52.67801833333333, -8.569161666666666)","(52.666845, -8.543201666666667)",4,160.655172,316.0,"(52.666845, -8.543201666666667)",...,...,...,NaN,School,Residential,Residential
4654,P666_ZZ-JG_87.0,3062.58,21.216667,2.405797,2023-09-20 16:22:05,2023-09-20 16:43:18,"(52.666205, -8.54393)","(52.66678666666667, -8.543111666666666)",5,119.760000,281.0,"(52.65831333333333, -8.54979)",...,...,...,NaN,Residential,Residential,School
4655,P666_ZZ-JG_88.0,2377.76,24.033333,1.648932,2023-09-20 17:28:30,2023-09-20 17:52:32,"(52.66665833333333, -8.543538333333334)","(52.66679666666667, -8.543175)",5,211.750000,581.0,"(52.66080333333333, -8.54976)",...,...,...,NaN,Residential,Residential,Residential
4656,P666_ZZ-JG_89.0,2975.14,16.933333,2.928287,2023-09-20 18:53:16,2023-09-20 19:10:12,"(52.66588166666666, -8.545276666666666)","(52.666855, -8.543191666666667)",6,72.350000,365.0,"(52.65833666666666, -8.549711666666667)",...,...,...,NaN,Recreational,Residential,Recreational


In [54]:
features_df['start_label'] = osm_data['start_category']
features_df['end_label'] = osm_data['end_category']
features_df['max_halt_label'] = osm_data['max_halt_category']
features_df

features_df['start_location'] = osm_data['start_location']
features_df['end_location'] = osm_data['end_location']
features_df['max_halt_location'] = osm_data['max_halt_location']
features_df

,trip_id,distance,duration,avg_speed,start_time,end_time,start_location,end_location,halt_count,avg_halt_duration,max_halt_duration,max_halt_location,start_label,end_label,max_halt_label
0,C001_06-Veloci2_1.0,5080.199201,19.950000,4.244110,2024-01-31 17:49:44,2024-01-31 18:09:41,"(52.66710166666667, -8.576183333333333)","(52.67005833333333, -8.633385)",6,33.791667,96.0,"(52.66534, -8.607245)",Commercial,Residential,Commercial
1,C001_06-Veloci2_110.0,6298.928335,19.500000,5.383699,2024-03-04 08:42:54,2024-03-04 09:02:24,"(52.67011, -8.633483333333333)","(52.667611666666666, -8.574975)",3,7.636364,20.0,"(52.66794166666666, -8.576695)",Residential,Commercial,Residential
2,C001_06-Veloci2_111.0,1765.016288,9.516667,3.091097,2024-03-04 10:18:56,2024-03-04 10:28:27,"(52.666605, -8.57684)","(52.672138333333336, -8.556015)",3,86.214286,309.0,"(52.672138333333336, -8.556015)",Recreational,Commercial,Commercial
3,C001_06-Veloci2_112.0,3399.361455,7.916667,7.156550,2024-03-04 12:21:25,2024-03-04 12:29:20,"(52.67209666666667, -8.555928333333334)","(52.671821666666666, -8.55602)",3,188.461538,451.0,"(52.671821666666666, -8.55602)",Commercial,Commercial,Commercial
4,C001_06-Veloci2_113.0,8880.359707,33.950000,4.359529,2024-03-04 18:16:24,2024-03-04 18:50:21,"(52.671821666666666, -8.55602)","(52.66999833333333, -8.63372)",8,41.325000,223.0,"(52.670008333333335, -8.633391666666666)",Commercial,Residential,Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4653,P666_ZZ-JG_86.0,4132.682274,21.433333,3.213594,2023-09-20 15:43:41,2023-09-20 16:05:07,"(52.67801833333333, -8.569161666666666)","(52.666845, -8.543201666666667)",4,160.655172,316.0,"(52.666845, -8.543201666666667)",School,Residential,Residential
4654,P666_ZZ-JG_87.0,3149.144739,21.216667,2.473798,2023-09-20 16:22:05,2023-09-20 16:43:18,"(52.666205, -8.54393)","(52.66678666666667, -8.543111666666666)",5,119.760000,281.0,"(52.65831333333333, -8.54979)",Residential,Residential,School
4655,P666_ZZ-JG_88.0,3692.867836,24.033333,2.560935,2023-09-20 17:28:30,2023-09-20 17:52:32,"(52.66665833333333, -8.543538333333334)","(52.66679666666667, -8.543175)",5,211.750000,581.0,"(52.66080333333333, -8.54976)",Residential,Residential,Residential
4656,P666_ZZ-JG_89.0,3149.814093,16.933333,3.100211,2023-09-20 18:53:16,2023-09-20 19:10:12,"(52.66588166666666, -8.545276666666666)","(52.666855, -8.543191666666667)",6,72.350000,365.0,"(52.65833666666666, -8.549711666666667)",Recreational,Residential,Recreational


In [57]:
features_df.to_csv('data/processed/trip_features_with_location_context.csv', index=False)